# Gensim based

In [ ]:
!pip install --upgrade gensim
!pip install nltk

In [ ]:
def get_article(path):
    with io.open(path, "r", encoding="utf-8") as f:
        article = f.read()
    return article

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from collections import defaultdict
from gensim import corpora
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import io

file = "a1.txt" 
contents = get_article(file)
documents = sent_tokenize(contents)

# data = []
# with open(file, 'r') as f:
#     contents = f.readlines()
#     for content in contents:

# remove common words and tokenize
stoplist = stopwords.words('english')
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from gensim import models
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=3)

In [ ]:
doc = "It was nominated for ten Academy Awards"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

[(0, -0.3587363809693009), (1, -0.23682321727431366), (2, 0.18275185255745596)]


In [ ]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it

In [ ]:
index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')
sims = index[vec_lsi]  # perform a similarity query against the corpus
print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

[(0, 0.40990523), (1, 0.9393433), (2, 0.055460922), (3, 0.44547313), (4, 0.45062095), (5, 0.4930768), (6, 0.91818047), (7, 0.9734668), (8, 0.4263637), (9, 0.050754245), (10, 0.04337981), (11, 0.99148685), (12, 0.21084052), (13, 0.026073815), (14, -0.28001755), (15, -0.32598287), (16, -0.25347245), (17, -0.29577604), (18, -0.3087811), (19, 0.10254729), (20, -0.22653693), (21, 0.5443905), (22, 0.026472237), (23, 0.040434383), (24, 0.021658419), (25, -0.28509477), (26, -0.28808728), (27, -0.20860463), (28, -0.16015886), (29, 0.044739183), (30, -0.28412512), (31, 0.0061076125), (32, -0.4692716), (33, -0.19946642), (34, -0.25561056), (35, 0.46209735), (36, -0.2658413), (37, -0.2831256), (38, -0.2989611), (39, -0.23263347), (40, -0.16929357), (41, -0.0074590063), (42, -0.098112606), (43, 0.23812978), (44, -0.29572907), (45, 0.0), (46, -0.25927466), (47, -0.43277934), (48, 0.80385315), (49, 0.04292325), (50, 0.20759907), (51, 0.04611678), (52, 0.085612826), (53, 0.941728), (54, 0.14719431), (

In [ ]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
cnt = 0
for doc_position, doc_score in sims:
  if cnt > 10: break
  cnt += 1
  print(doc_score, documents[doc_position])

0.9985411 At the 84th Academy Awards, The Artist received ten nominations, winning five awards, including Best Picture, Best Director for Hazanavicius, Best Actor in a Leading Role for Jean Dujardin, Best Costume Design, and Best Original Score.
0.99148685 In France it was nominated for ten César Awards, winning six, including Best Film, Best Director for Hazanavicius and Best Actress for Bejo.
0.9734668 It was nominated for ten Academy Awards and won five, including Best Picture for Langmann, Best Director for Hazanavicius, and Best Actor for Dujardin, making him the first French actor ever to win for Best Actor.
0.941728 Hazanavicius chose the form of the melodrama, mostly because he thought many of the films from the silent era which have aged best are melodramas.
0.9393433 It was written, directed, and co-edited by Michel Hazanavicius, produced by Thomas Langmann and starred Jean Dujardin and Bérénice Bejo.
0.91818047 In January 2012, the film was nominated for twelve BAFTAs, the m

In [ ]:
!pip install --upgrade gensim
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
!pip install --upgrade gensim
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-50")
model.most_similar("implementation")

[('implementing', 0.9255009293556213),
 ('implement', 0.9029074907302856),
 ('implemented', 0.8602141737937927),
 ('framework', 0.8587372303009033),
 ('comprehensive', 0.81963050365448),
 ('process', 0.8084008097648621),
 ('facilitate', 0.7937219142913818),
 ('mechanism', 0.7825443744659424),
 ('verification', 0.7752116322517395),
 ('consultation', 0.7690036296844482)]

# BM25 AND SBERT

In [ ]:
!pip install sentence_transformers

In [71]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
s1 = "He is the best idol in the world."
s2 = "His name is GoodBai, a Chinese actor and rapper."
s3 = "GoodBai is a really handsome and encouraging actor."
sentence_embedding = model.encode([s1, s2, s3])
print(sentence_embedding)

[[-0.24911211  0.32584146  0.93652743 ... -0.48911288  0.55232924
  -0.27428582]
 [ 0.24896492  0.70574325 -0.17513533 ...  0.9390527   0.28650507
  -1.0451123 ]
 [-0.21866459  0.53278804  0.60553914 ...  0.08831918  0.5542333
  -0.57935375]]


In [ ]:
# credit https://www.youtube.com/watch?v=ziiF1eFM3_4
!pip install numpy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

scores = np.zeros((sentence_embedding.shape[0], sentence_embedding.shape[0]))
for i in range(sentence_embedding.shape[0]):
  scores[i, :] = cosine_similarity([sentence_embedding[i]], sentence_embedding)[0]

print(scores)


[[1.         0.40182683 0.74102485]
 [0.40182683 0.99999988 0.4754245 ]
 [0.74102485 0.47542444 1.00000012]]


In [83]:
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# !pip install -U spacy==2.1.0
# !python -m spacy download en
# !pip uninstall -y neuralcoref 
# !pip install neuralcoref --no-binary neuralcoref
# import spacy
# import neuralcoref

# Load the model
# nlp = spacy.load('en_core_web_sm')  
# neuralcoref.add_to_pipe(nlp)

q1 = "The genre of the film is something."
q2 = "The box office of the film is something."
q3 = "The film is criticized for something."
q4 = "Some characters in the film are someththing."
q5 = "The film was released sometime."
q6 = "The director of the film is someone."

q1 = "What is the genre of the film?"
q2 = "What is the box office of the film?"
q3 = "What is the film is criticized for?"
q4 = "What are some characters in the film?"
q5 = "When was the film released?"
q6 = "Who is the director of the film?"
sentences = []
sentences += [q1, q2, q3, q4, q5, q6]

punctuations = '!"#$%&\'*,.:;?~'

with open("a1.txt", 'r') as f:
  for sent in f.readlines():
    if sent == "\n":
      continue
    else:
      para = sent.replace("\n", "")
      # doc = nlp(para)
      # para = doc._.coref_resolved
      # candidates = sent_tokenize(para)
      # for cand in candidates:
        # if cand[-1] in punctuations:
        #     sentences.append(cand)
      if para[-1] in punctuations:
          sentences.append(para)
embeddings = model.encode(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# heap testing
import heapq
h = []
heapq.heapify(h)
heapq.heappush(h, (1, "one"))
heapq.heappush(h, (20, "twenty"))
heapq.heappush(h, (-1, "neg one"))
print(1, heapq.heappop(h))
print(2, heapq.heappop(h))
print(3, heapq.heappop(h))

In [95]:
import heapq
from sklearn.metrics.pairwise import cosine_similarity

msize = 5
for idx in range(5,6):
  print(sentences[idx], "\n")
  size = 0
  pq = []
  heapq.heapify(pq)

  print("\nQ{idx}: \n".format(idx = idx))
  for (i, res) in enumerate(cosine_similarity([embeddings[idx]], embeddings)[0]):
    if i < 7: continue
    score = res
    heapq.heappush(pq, (score, sentences[i]))
    size += 1
    if size > msize:
      heapq.heappop(pq)

  while pq:
    print(heapq.heappop(pq))


Who is the director of the film? 


Q5: 

(0.34616894, "Director Michel Hazanavicius had been fantasizing about making a silent film for many years, both because many filmmakers he admires emerged in the silent era, and because of the image-driven nature of the form. According to Hazanavicius his wish to make a silent film was at first not taken seriously, but after the financial success of his spy-film pastiches OSS 117: Cairo, Nest of Spies and OSS 117: Lost in Rio, producers started to express interest. The forming of the film's narrative started with Hazanavicius' desire to work again with actors Jean Dujardin and Bérénice Bejo, Hazanivicius's wife, who had both starred in OSS 117: Cairo, Nest of Spies. Hazanavicius chose the form of the melodrama, mostly because he thought many of the films from the silent era which have aged best are melodramas. He did extensive research about 1920s Hollywood, and studied silent films to find the right techniques to make the story comprehensible 